In [ ]:
import cv2
import cvzone # we need to used this para ma highlight yung mga objectct o yung shape na makikita sa camera
import numpy as np

cap = cv2.VideoCapture("http://192.168.254.117:4747/video")  
# specificy yung height and width ng iyong camera
cap.set(3,640)
cap.set(4,480)
totalCount = 0
def empty(a):
    pass 
# ginagamit ko to for realtime adjustment ng mga threshold values
cv2.namedWindow("Setting")  
cv2.resizeWindow("Setting", 640, 240) 
cv2.createTrackbar("Threshold1", "Setting", 24, 255,empty )  # Threshold1 for Canny
cv2.createTrackbar("Threshold2", "Setting", 82, 255,empty )

def preProcess(img):
   
   imgPre = cv2.GaussianBlur(img,(11,11),3) # gagawing japan mode masmaganda daw pag blur 
   threshold1 = cv2.getTrackbarPos("Threshold1", "Setting")  
   threshold2 = cv2.getTrackbarPos("Threshold2", "Setting")  
   imgPre = cv2.Canny(imgPre, threshold1 , threshold2) # parang dradrawing mo yung shape ng object mo 
   kernel = np.ones((3,3),np.uint8)
   imgPre = cv2.dilate(imgPre, kernel, iterations=1)  #to make edges thicker yung coin
   imgPre = cv2.morphologyEx(imgPre, cv2.MORPH_CLOSE, kernel)  # to close small holes in the edges ng mga coin 



   return imgPre
# basically it capture frame by frame of your camera diba english 
while True:
    sucess, img = cap.read()
    imgPre = preProcess(img)
    imgContours, conFound = cvzone.findContours(img, imgPre, minArea=20)  # find contours in the preprocessed image
    totalCount = 0  # reset total count for each frame
    if conFound:
        for contour in conFound:    #getting yung area ng iyong coin gamit yung contour
            peri = cv2.arcLength(contour['cnt'], True)
            approx = cv2.approxPolyDP(contour['cnt'], 0.02 * peri, True)

           
            if len(approx)>5:
                area = (contour['area'])

                if area<12830:
                    totalCount += 1
                elif 15940<area<16250:
                    totalCount += 5
                else:
                    totalCount += 20
                
    print(totalCount)

    if sucess:
          
        imgStacked = cvzone.stackImages([img, imgPre,imgContours], 2, 1) # pagsamahin ko yung original image at yung preprocessed image
        cvzone.putTextRect(imgStacked, f'Total: {totalCount}', (50, 50))
        cv2.imshow("Image", imgStacked)    # parang system out print sa java 
    else:
        print("Failed to capture image")
    cv2.waitKey(1)